# Question 3, work in progress

# Assumption of no clouds

In [17]:
# SHORT WAVE

P_sol= 341.3
r_SM = 0.1065

a_O3 = 0.08

a_SW = 0.1451

r_SE = 0.17

P_refl_atm = r_SM*P_sol

P_abs_atm = (a_SW+ a_O3)*(1-r_SM)*P_sol

#print(P_refl_atm, P_abs_atm)

P_trans_atm = (1 - (a_SW + a_O3))*(1- r_SM)*P_sol


P_refl_earth_total = (r_SE*P_trans_atm)/(1-r_SE*r_SM)

P_abs_earth_total = (P_trans_atm)/(1-r_SE*r_SM)




P_refl_atm_total =r_SM*(r_SE*P_trans_atm)/(1-r_SM*r_SE)

P_abs_atm_total =  (a_SW+a_O3)*(1-r_SM)*P_refl_earth_total


print(P_refl_earth_total, P_abs_earth_total, P_refl_atm_total, P_abs_atm_total)
print(P_refl_earth_total + P_abs_earth_total + P_refl_earth_total - (1-(a_O3+ a_SW)*(1-r_SM)*P_refl_earth_total))


# LONG WAVE

sigma = 5.67*10**(-8) #W m-2 K-4

epsilon_A = 0.875

f_A = 0.618


alpha = 3
beta = 4

a_LW = 0.8258

#P_E0 = sigma*T_E**4
#P_A0 = epsilon_A*sigma*T_A**4
#P_A0_down = f_A*P_A0
#P_A0_up = (1-f_A)*P_A0
#P_EtoA = - alpha*(T_A -T_E)
#P_latent = -beta*(T_A-T_E)



from scipy.optimize import fsolve

def eqns(variables):
    T_E, T_A = variables

    P_non_rad = (alpha + beta) * (T_E - T_A)
    #               P_abs_earth   +     P_A0_down    -    (P_EtoA + P_latent)    - P_E0
    eqn1 = P_abs_earth_total + epsilon_A*sigma* T_A**4 - P_non_rad - sigma* T_E**4
    #     P_abs_atm  +       a_LW*P_E0    +  (P_EtoA + P_latent)     - P_A0
    eqn2 = (P_refl_earth_total) + a_LW*sigma* T_E**4 + P_non_rad - (1+f_A)*epsilon_A*sigma* T_A**4 - (1 - (a_SW + a_O3))*(1- r_SM)*P_refl_earth_total

    return [eqn1, eqn2]

T_init_guess = [298.0, 278.0]
solution, info , ier, mesg = fsolve(eqns, T_init_guess, xtol=1e-8, full_output=True)

print("Solution: ", solution)





40.91291078592925 240.66418109370144 4.357224998701465 8.228684870704976
329.7186875362649
Solution:  [293.15514942 276.92010785]


In [16]:
solution - 273 # Celsius

array([20.15514942,  3.92010785])

In [18]:
# SHORT WAVE

P_sol = 341.3
r_SM = 0.1065
a_O3 = 0.08
a_SW = 0.1451
r_SE = 0.17

# Initial Reflections and Absorptions
P_refl_atm_initial = r_SM * P_sol  # Initial reflection by atmosphere
P_abs_atm_initial = (a_SW + a_O3) * (1 - r_SM) * P_sol  # Initial absorption by atmosphere
P_trans_atm_initial = (1 - (a_SW + a_O3)) * (1 - r_SM) * P_sol  # Transmitted to Earth

# Calculate Total Reflected Energy using Geometric Series Formula
R_total = (r_SM + r_SE - 2 * r_SM * r_SE) / (1 - r_SM * r_SE)
P_ref_total = R_total * P_sol

# Total Absorbed Energy
P_abs_total = P_sol - P_ref_total

# Distribute Absorbed Energy
P_abs_atm_total = P_abs_atm_initial  # 68.6 W/m²
P_abs_earth_total = P_abs_total - P_abs_atm_total  # 257.9 -68.6≈189.3 W/m²

print(f"Total Reflected to Space (P_ref_total): {P_ref_total:.2f} W/m²")
print(f"Total Absorbed by Atmosphere (P_abs_atm_total): {P_abs_atm_total:.2f} W/m²")
print(f"Total Absorbed by Earth (P_abs_earth_total): {P_abs_earth_total:.2f} W/m²")
print(f"Sum of Reflected and Absorbed: {P_ref_total + P_abs_atm_total + P_abs_earth_total:.2f} W/m²")
# LONG WAVE

from scipy.optimize import fsolve

sigma = 5.67e-8  # W/m²/K⁴
epsilon_A = 0.875
f_A = 0.618
alpha = 3
beta = 4
a_LW = 0.8258


def eqns(variables):
    T_E, T_A = variables

    # Heat Fluxes
    P_non_rad = (alpha + beta) * (T_E - T_A)

    # Emissions
    P_E0 = sigma * T_E**4
    P_A0 = epsilon_A * sigma * T_A**4

    # Equation 1: Earth Energy Balance
    eqn1 = P_abs_earth_total + P_A0 - P_non_rad - P_E0

    # Equation 2: Atmosphere Energy Balance
    eqn2 = P_abs_atm_total + a_LW * P_E0 + P_non_rad - (1 + f_A) * P_A0

    return [eqn1, eqn2]

# Initial guesses
T_init_guess = [288.0, 290.0]  # K

# Solve the system
solution, info, ier, mesg = fsolve(eqns, T_init_guess, xtol=1e-8, full_output=True)

if ier == 1:
    T_E, T_A = solution
    print("Solution: ", solution)
    print(f"Equilibrium Earth Temperature (T_E): {T_E:.2f} K")
    print(f"Equilibrium Atmosphere Temperature (T_A): {T_A:.2f} K")
else:
    print("Solution not found:", mesg)

print(solution - 273)

Total Reflected to Space (P_ref_total): 83.52 W/m²
Total Absorbed by Atmosphere (P_abs_atm_total): 68.64 W/m²
Total Absorbed by Earth (P_abs_earth_total): 189.13 W/m²
Sum of Reflected and Absorbed: 341.30 W/m²
Solution:  [291.24627207 279.3485702 ]
Equilibrium Earth Temperature (T_E): 291.25 K
Equilibrium Atmosphere Temperature (T_A): 279.35 K
[18.24627207  6.3485702 ]


# Assumption with clouds

In [3]:
CC = 0.66
r_SC = 0.22
r_LC = 0.195
a_LC = 0.622
a_SC = 0.1239

                # atm refect    + clouds reflect
P_refl_atm = r_SM*(1-CC)*P_sol + r_SC*CC*P_sol
print(P_refl_atm)
                #
P_abs_atm =  (a_O3 + a_SW)*(1-CC)*(1 -r_SM)*P_sol  + (a_O3 + a_SC)*(1-r_SC)*CC*P_sol
print(P_abs_atm)

P_trans_atm = P_sol - P_refl_atm - P_abs_atm
print(P_trans_atm)

P_refl_earth = r_SE * P_trans_atm
P_abs_earth = (1 - r_SE)*P_trans_atm

a_LW_total = a_LW*(1-CC) + a_LC*CC
epsilon_A_total = epsilon_A*(1-CC) + a_LC*CC

def eqns(variables):
    T_E, T_A = variables

    P_non_rad = - (alpha + beta) * (T_A - T_E)
    #               P_abs_earth   +     P_A0_down    -    (P_EtoA + P_latent)    - P_E0
    eqn1 = P_abs_earth + f_A*epsilon_A_total*sigma* T_A**4 + P_non_rad - sigma* T_E**4

    #     P_abs_atm  +       a_LW*P_E0    +  (P_EtoA + P_latent)     - P_A0
    eqn2 = P_abs_atm + a_LW_total*sigma* T_E**4 - P_non_rad - epsilon_A_total*sigma* T_A**4

    return [eqn1, eqn2]

T_init_guess = [298.0, 288.0]
solution, info , ier, mesg = fsolve(eqns, T_init_guess, xtol=1e-8, full_output=True)

print("Solution: ", solution)



61.915233
59.16464476370001
220.2201222363
Solution:  [300.61406609 283.4579788 ]


In [4]:
solution - 273

array([27.61406609, 10.4579788 ])